# Evaluate reasoning datasets with `logikon` debuggers

Use this notebook to

* debug a reasoning dataset with `logikon`
* customize `logikon` configuration (e.g., expert_model, debuggers)
* upload debugging results (artifacts and metrics) to huggingface hub

### Set-up

In [1]:
import getpass
import os

GH_ACCESS_TOKEN = getpass.getpass("GH_ACCESS_TOKEN: ")
if GH_ACCESS_TOKEN:
    os.environ["GH_ACCESS_TOKEN"] = GH_ACCESS_TOKEN

OPENAI_API_KEY = getpass.getpass("OPENAI_API_KEY: ")
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

HUGGINGFACEHUB_API_TOKEN = getpass.getpass("HUGGINGFACEHUB_API_TOKEN: ")
if HUGGINGFACEHUB_API_TOKEN:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [2]:
# install logikon module from github repo

# hatch packaging
%pip install hatch

# for using local models with vllm / example 3:
%pip install -U "vllm>=0.1.7" git+https://$GH_ACCESS_TOKEN@github.com/logikon-ai/logikon.git
%pip install "tokenizers==0.13.3"
%pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 103.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 101.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 20.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-

In [3]:
%pip list | grep pydantic

pydantic                          1.10.12
Note: you may need to restart the kernel to use updated packages.


### Imports, data models, configs

In [2]:
import dataclasses
import datetime
import json
import os
import pprint
import tempfile
import uuid

import datasets
import huggingface_hub as hf_hub
import logikon
import networkx as nx

print(f"Installed `logikon` module version: {logikon.__version__}")

Installed `logikon` module version: 0.0.1.dev0


In [3]:
@dataclasses.dataclass
class ReferenceDataset:
    """Reference dataset metadata."""
    path: str
    id_field: str = None
    split: str = None
    revision: str = None

@dataclasses.dataclass
class EvalResultRecord:
    """Evaluation result record."""
    id: str
    reference_id: str
    reference_dataset: dict
    lang: str
    created_date: str
    argmap: dict
    scores: dict
    model: str
    metadata: dict

In [31]:
REASONING_DATASET = dict(
    path='logikon/oasst1-delib',
    split='validation',
    revision=None,
    id_field='message_id',
)

EVALRESULTS_DATASET_PATH = 'logikon/delib-evals'

LANG = "en"

DEBUG_CONFIG = dict(
    artifacts=["networkx_graph"],
    metrics=["argmap_size", "argmap_avg_katz_centrality","argmap_attack_ratio"],
    expert_model="Open-Orca/OpenOrca-Platypus2-13B",
    llm_framework="VLLM",
    #expert_model="gpt-3.5-turbo-instruct",
    #llm_framework="OpenAI",
)

In [32]:
reference_dataset = ReferenceDataset(**REASONING_DATASET)
config = logikon.DebugConfig(**DEBUG_CONFIG)
evalresults_dataset = dict(
    path=EVALRESULTS_DATASET_PATH,
    filename=f"{config.expert_model.replace('/','_')}_{reference_dataset.split}.jsonl",
    subfolder=os.path.join("data",reference_dataset.path),
)

In [33]:
print(reference_dataset)
print(config)
print(evalresults_dataset)

ReferenceDataset(path='logikon/oasst1-delib', id_field='message_id', split='validation', revision=None)
expert_model='Open-Orca/OpenOrca-Platypus2-13B' expert_model_kwargs={'temperature': 0.7} llm_framework='VLLM' generation_kwargs=None inputs=[] metrics=['argmap_size', 'argmap_avg_katz_centrality', 'argmap_attack_ratio'] artifacts=['networkx_graph'] report_to=[]
{'path': 'logikon/delib-evals', 'filename': 'Open-Orca_OpenOrca-Platypus2-13B_validation.jsonl', 'subfolder': 'data/logikon/oasst1-delib'}


### Load reference dataset

In [34]:
# login to huggingface hub
hf_hub.login(os.environ["HUGGINGFACEHUB_API_TOKEN"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [38]:
# load the reference dataset
ref_ds = datasets.load_dataset(
    reference_dataset.path,
    split=reference_dataset.split,
    revision=reference_dataset.revision,
)
ref_ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6 [00:00<?, ? examples/s]

Dataset({
    features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels', 'history'],
    num_rows: 6
})

In [9]:
# optional: check ref dataset features
# ref_ds.features

In [39]:
assert reference_dataset.id_field in ref_ds.features, f"Reference dataset must have {reference_dataset.id_field} field."

In [40]:
# download the eval results dataset (if exists)
eval_results = []

if hf_hub.file_exists(
    evalresults_dataset["path"], os.path.join(evalresults_dataset["subfolder"], evalresults_dataset["filename"]),
    repo_type="dataset",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
):
        
    eval_file = hf_hub.hf_hub_download(
        evalresults_dataset["path"],
        filename=evalresults_dataset["filename"],
        subfolder=evalresults_dataset["subfolder"],
        repo_type="dataset"
    )

    with open(eval_file) as f:
        for line in f:
            record = EvalResultRecord(**json.loads(line))
            eval_results.append(record)

    print(f"Loaded {len(eval_results)} eval results.")
    


In [41]:
# iterate over the reference dataset:
# - compute the metrics for records not in the eval results dataset
# - update the eval results dataset

for example in ref_ds:

    if any(r.reference_id == example[reference_dataset.id_field] for r in eval_results):
        print(f"Skipping {example[reference_dataset.id_field]}")
        continue

    print("scoring example:")
    debug_results = logikon.score(
        prompt=example["history"],
        completion=example["text"],
        config=config
    )

    argmap = next((a.data for a in debug_results.artifacts if a.id == "networkx_graph"), None)
    argmap = nx.node_link_data(argmap) if argmap else {}

    scores = {
        "id": [],
        "value": [],
        "comment": [],
    }
    for score in debug_results.scores:
        for key in scores:
            scores[key].append(score.__getattribute__(key))
    scores["names"] = scores.pop("id")
    scores["values"] = scores.pop("value")
    scores["comments"] = scores.pop("comment")

    eval_record = EvalResultRecord(
        id = str(uuid.uuid4()),
        reference_id=example[reference_dataset.id_field],
        reference_dataset=dataclasses.asdict(reference_dataset),
        lang=example.get("lang", LANG),
        created_date=datetime.datetime.now().isoformat(),
        argmap=argmap,
        scores=scores,
        model=config.expert_model,
        metadata={"logikon_config": config.dict(), "logikon_version": logikon.__version__},
    )

    eval_results.append(eval_record)

scoring example:
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

INFO 09-27 14:32:58 llm_engine.py:72] Initializing an LLM engine with config: model='Open-Orca/OpenOrca-Platypus2-13B', tokenizer='Open-Orca/OpenOrca-Platypus2-13B', tokenizer_mode=auto, trust_remote_code=True, dtype=torch.float16, download_dir=None, load_format=auto, tensor_parallel_size=1, seed=0)


INFO 09-27 14:36:46 llm_engine.py:199] # GPU blocks: 1469, # CPU blocks: 327


> Entering new ClaimExtractionChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment

Identify the basic question addressed in a text.

# Detailed Instructions

Use the inputs (a TEXT) to solve your assignment and read the following instructions carefully.
Identify the overarching question the TEXT raises and addresses.
Note that the overarching question is not necessarily explicitly stated in the TEXT (as shown in some examples below). It may be implicit. And it may deviate from any explicitly stated questions or instructions.
State a single bold question in a concise and most-simple way. Use plain language.
Don't provide alternatives, comments or explanations.

# Example

TEXT:
:::
Should we visit Mars?
- Mars

...


> Finished chain.
> Answer: 

- You could have difficulty understanding speech, especially in noisy environments.
- You could have difficulty concentrating or sleeping.
- You could experience tinnitus, which is a ringing or buzzing sound in the ears.

The first quote highlights the claim about experiencing tinnitus. However, please note that the claim is not the only one mentioned in the quote, so please choose the most relevant part.

- You could experience tinnitus, which is a ringing or buzzing sound in the ears.

This is the most succinct and clear quote related to the claim about experiencing tinnitus.

# Verbatim Quote
You could experience tinnitus, which is a ringing or buzzing sound in the ears.


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment
Find a telling title for an argument.

# Inpu

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.78it/s]



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment
Identify the text-span in a text that expresses a given claim most clearly.

# Inputs
Use the following inputs (a TEXT and a certain CLAIM) to solve your assignment.

TEXT:
:::
Pros of lowering the capabilities of your ears by listening to loud music (70-75db) in order to withstand the incredibly loud sound of screaming kids at school:

- You may be able to block out the sound of the screaming kids, which could make it easier to concentrate or relax.
- You may be able to develop a tolerance to loud noise, which could make it less unpleasant to be around screaming kids.

Cons of lowering the capabilities of your ears by listening to loud music (70-75db) in order to withstand the incredibly loud sound of screaming kids at school:

-

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.74s/it]


> Finished chain.
> Answer: 

"- You could have difficulty understanding speech, especially in noisy environments."

This text-span most clearly presents the claim about having difficulty understanding speech, especially in noisy environments. It is a direct statement of the consequence of lowering the capabilities of your ears by listening to loud music.

Please note that although there is only one answer, the provided text contains several claims. You asked for the most clear statement of the claim in question, which is why I chose the text-span shown above.
###
Mining arguments at depth 2.
 Number of target nodes: 0.
 Number of nodes in argmap: 1.
###

> Finished chain.


In [42]:
for record in eval_results:
    print(json.dumps(dataclasses.asdict(record)))

{"id": "f4d6bc51-1213-409a-a0ff-1fdadf768cbf", "reference_id": "c743da26-28dd-45fc-8a52-e1bd500ca8a2", "reference_dataset": {"path": "logikon/oasst1-delib", "id_field": "message_id", "split": "validation", "revision": null}, "lang": "en", "created_date": "2023-09-27T14:38:08.445112", "argmap": {"directed": true, "multigraph": false, "graph": {}, "nodes": [{"text": "TensorFlow is an open-source machine learning library developed by Google.", "label": "Claim-1", "annotations": [], "nodeType": "proposition", "id": "42cc8358-e2d7-427c-86fe-b604470f5b4d"}, {"text": "PyTorch is an open-source machine learning library maintained by Facebook.", "label": "Claim-2", "annotations": [], "nodeType": "proposition", "id": "a943d165-3784-42bd-af22-94fafc62ce58"}, {"text": "Keras is a high-level neural networks API written in Python.", "label": "Claim-3", "annotations": [], "nodeType": "proposition", "id": "4f06eadd-a0c6-4ba2-9267-224485dcc44b"}, {"text": "MXNet is a flexible and efficient deep learnin

In [43]:
# save the eval results dataset

with tempfile.NamedTemporaryFile() as tmpf:
    for record in eval_results:
        tmpf.write(json.dumps(dataclasses.asdict(record)).encode("utf-8"))
        tmpf.write("\n".encode("utf-8"))
    tmpf.flush()
    
    api = hf_hub.HfApi()
    api.upload_file(
        path_or_fileobj=tmpf.name,
        path_in_repo=os.path.join(evalresults_dataset["subfolder"],evalresults_dataset["filename"]),
        repo_id=evalresults_dataset["path"],
        repo_type="dataset",
        token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    )


In [45]:
# check whether eval results have been uploaded

ds_test = datasets.load_dataset(evalresults_dataset["path"])
ds_test

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'reference_id', 'reference_dataset', 'lang', 'created_date', 'argmap', 'scores', 'model', 'metadata'],
        num_rows: 3
    })
})

In [46]:
ds_test["validation"].features

{'id': Value(dtype='string', id=None),
 'reference_id': Value(dtype='string', id=None),
 'reference_dataset': {'path': Value(dtype='string', id=None),
  'id_field': Value(dtype='string', id=None),
  'split': Value(dtype='string', id=None),
  'revision': Value(dtype='null', id=None)},
 'lang': Value(dtype='string', id=None),
 'created_date': Value(dtype='string', id=None),
 'argmap': {'directed': Value(dtype='bool', id=None),
  'multigraph': Value(dtype='bool', id=None),
  'graph': {},
  'nodes': [{'text': Value(dtype='string', id=None),
    'label': Value(dtype='string', id=None),
    'annotations': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None),
    'nodeType': Value(dtype='string', id=None),
    'id': Value(dtype='string', id=None)}],
  'links': [{'valence': Value(dtype='string', id=None),
    'source': Value(dtype='string', id=None),
    'target': Value(dtype='string', id=None)}]},
 'scores': {'names': Sequence(feature=Value(dtype='string', id=None), length=-1, i